In [1]:
import ast
import copy
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from aocd import get_data, submit

DAY = 23
YEAR = 2022

In [123]:
# use test data
raw_test = """..............
..............
.......#......
.....###.#....
...#...#.#....
....#...##....
...#.###......
...##.#.##....
....#..#......
..............
..............
.............."""

# use real data
raw = get_data(day=DAY, year=YEAR)

print(raw_test)

..............
..............
.......#......
.....###.#....
...#...#.#....
....#...##....
...#.###......
...##.#.##....
....#..#......
..............
..............
..............


In [137]:
def parse_data(data):
    elves = set()
    data = data.split("\n")
    for rdx, row in enumerate(data):
        for cdx, el in enumerate(list(row)):
            if el == "#":
                elves.add((cdx + 1, rdx + 1))

    return elves


dummy = parse_data(raw_test)
real = parse_data(raw)

dummy

{(4, 5),
 (4, 7),
 (4, 8),
 (5, 6),
 (5, 8),
 (5, 9),
 (6, 4),
 (6, 7),
 (7, 4),
 (7, 7),
 (7, 8),
 (8, 3),
 (8, 4),
 (8, 5),
 (8, 7),
 (8, 9),
 (9, 6),
 (9, 8),
 (10, 4),
 (10, 5),
 (10, 6),
 (10, 8)}

# Part 1

In [206]:
def consider_positions_single(grid, coords):
    global dir_groups
    global dir_offset
    global directions

    cx, cy = coords
    elves_nb = {(cx + x, cy + y) for x, y in directions.values() if (cx + x, cy + y) in grid}

    if len(elves_nb) == 0:
        return coords

    valid_move = False
    for idx in range(dir_offset, dir_offset + 4):
        idx = idx % 4
        if all([(cx + x, cy + y) not in grid for x, y in dir_groups[idx]]):
            valid_move = True
            break

    if valid_move:
        x, y = dir_groups[idx][1]
        return (cx + x, cy + y)

    return coords


def consider_positions(grid):
    global dir_offset

    moves = {}
    for idx, elf in enumerate(grid):
        new_coords = consider_positions_single(grid, elf)
        if new_coords not in moves:
            moves[new_coords] = []
        moves[new_coords].append(elf)

    return moves


def execute_positions(grid):
    global dir_offset

    proposed_moves = consider_positions(grid)
    new_moves = set()

    for new_poz, elf_list in proposed_moves.items():
        if len(elf_list) == 1:
            new_moves.add(new_poz)
        else:
            for elf in elf_list:
                new_moves.add(elf)
    return new_moves


def eval_grid(grid, plot=False):
    grid = np.array(list(grid))
    xmin, ymin = grid.min(axis=0)
    xmax, ymax = grid.max(axis=0)
    mask = np.zeros((ymax - ymin + 1, xmax - xmin + 1))
    for x, y in grid:
        mask[y - ymin, x - xmin] = 1

    if plot:
        plt.imshow(mask, extent=[xmin - 0.5, xmax + 0.5, ymax + 0.5, ymin - 0.5])
        for x, y in grid:
            plt.text(x, y, f"{(x,y)}", ha="center", va="center", fontsize=7)

    return np.count_nonzero(1 - mask)


grid = copy.deepcopy(real)

directions = {
    "N": [0, -1],
    "S": [0, 1],
    "W": [-1, 0],
    "E": [1, 0],
    "NE": [1, -1],
    "SE": [1, 1],
    "SW": [-1, 1],
    "NW": [-1, -1],
}
dir_groups = [["NW", "N", "NE"], ["SW", "S", "SE"], ["NW", "W", "SW"], ["NE", "E", "SE"]]
dir_groups = [[directions[d] for d in group] for group in dir_groups]
dir_offset = 0

rounds = 10
while rounds > 0:
    grid = execute_positions(grid)
    dir_offset += 1
    rounds -= 1

result = eval_grid(grid)
result

4172

In [173]:
# submit(result, part="a", day=DAY, year=YEAR)

# Part 2

In [208]:
grid = copy.deepcopy(real)

rounds = 1
dir_offset = 0
while True:
    new_grid = execute_positions(grid)
    if new_grid == grid:
        break
    grid = new_grid
    dir_offset += 1
    rounds += 1

result = rounds
result

942

In [209]:
# submit(result, part="b", day=DAY, year=YEAR)